In [ ]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
from llm.prompt_loader import PromptLoader
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from llm.ml_tools import svd_embed_score
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/home/hallscope/output/pope_val')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely
0,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a bear in the image?\nAnswer the ques...,1,119,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a car in the image?\nAnswer the quest...,0,1366,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
2,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a dining table in the image?\nAnswer ...,0,2096,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
3,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a car in the image?\nAnswer the quest...,0,730,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
4,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a car in the image?\nAnswer the quest...,0,1750,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [5]:
gt_label=df['label'].to_numpy()

In [ ]:
length=len(data)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 500
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
gt_label_test = []
gt_label_wild = []
gt_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
gt_label_test = gt_label[test_idxs]
gt_label_wild = gt_label[train_idxs]
gt_label_val = gt_label[val_idxs]

print(f'trainset length:{len(gt_label_wild)}')
print(f'validset length:{len(gt_label_val)}')
print(f'testset length:{len(gt_label_test)}')
print(f'trainset postive: {sum(gt_label_wild==1)} negative: {sum(gt_label_wild==0)}')
print(
    f'testset postive: {sum(gt_label_test==1)} negative: {sum(gt_label_test==0)}')

trainset length:1900
validset length:500
testset length:600
trainset postive: 933 negative: 967
testset postive: 301 negative: 299


In [ ]:
all_embeddings = df['most_likely'].apply(lambda x: np.squeeze(x['embedding'])).to_list()
all_embeddings =np.stack(all_embeddings)
all_embeddings.shape

(3000, 33, 4096)

In [ ]:
embedding_train = all_embeddings[train_idxs][:, 1:, :]
embedding_val = all_embeddings[val_idxs][:, 1:, :]
embedding_test = all_embeddings[test_idxs][:, 1:, :]

In [9]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

Grid Search for Best 'n_components' & 'layer'

In [10]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [ ]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,gt_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5199697962855857
found best n_components:1,layer:1 Score: 0.5260266049739734
found best n_components:1,layer:17 Score: 0.8423783818520661
found best n_components:1,layer:20 Score: 0.8651757599126021
found best n_components:1,layer:21 Score: 0.8890816785553628
found best n_components:1,layer:22 Score: 0.9199280251911831


  9%|▉         | 1/11 [00:00<00:08,  1.24it/s]

found best n_components:1,layer:26 Score: 0.9333911702332754
found best n_components:1,layer:27 Score: 0.9391427286164128
found best n_components:1,layer:30 Score: 0.9419060471692051


100%|██████████| 11/11 [00:15<00:00,  1.41s/it]


In [12]:
best_paras

{'n_components': 1, 'layer': 30, 'best_score': np.float64(0.9419060471692051)}

In [13]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [ ]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(gt_label_val)

In [ ]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(gt_label_wild,y_train)} AUROC:{roc_auc_score(gt_label_wild,scores)}')

train set acc: 0.8168421052631579 AUROC:0.9418007539256339


In [ ]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(gt_label_test,preds)} AUROC:{roc_auc_score(gt_label_test,scores)}')

test set acc: 0.87 AUROC:0.9495661062900699


Train Linear Prob to Get Better Results

In [ ]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=gt_label_test

In [18]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [19]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.8366666666666667 AUROC:0.864526272514139
